In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.set_option('display.max_colwidth', None)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import load_model
import re
from IPython.display import clear_output
import pickle

In [11]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


In [12]:
data = pd.read_csv("data/Sentiment.csv")
print(data.head())
to_keep = ['text', 'sentiment']
data = data[to_keep]

   id               candidate  candidate_confidence relevant_yn  \
0   1  No candidate mentioned                   1.0         yes   
1   2            Scott Walker                   1.0         yes   
2   3  No candidate mentioned                   1.0         yes   
3   4  No candidate mentioned                   1.0         yes   
4   5            Donald Trump                   1.0         yes   

   relevant_yn_confidence sentiment  sentiment_confidence     subject_matter  \
0                     1.0   Neutral                0.6578  None of the above   
1                     1.0  Positive                0.6333  None of the above   
2                     1.0   Neutral                0.6629  None of the above   
3                     1.0  Positive                1.0000  None of the above   
4                     1.0  Positive                0.7045  None of the above   

   subject_matter_confidence candidate_gold  ... relevant_yn_gold  \
0                     1.0000            NaN  ..

In [13]:
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about the Climate Change question last night? Exactly. #GOPDebate,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…,Positive
2,RT @TJMShow: No mention of Tamir Rice and the #GOPDebate was held in Cleveland? Wow.,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending -- hours after HER debate -- above any of the men in just-completed #GOPdebate says she's on …,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump delivered the highest ratings in the history of presidential debates. #Trump2016 http://t.co…,Positive


In [14]:
data['sentiment'].value_counts()

Negative    8493
Neutral     3142
Positive    2236
Name: sentiment, dtype: int64

In [15]:
data = data[data['sentiment'] != 'Neutral']
print(data.head())
data['sentiment'].value_counts()

                                                                                                                                           text  \
1  RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…   
3  RT @RobGeorge: That Carly Fiorina is trending -- hours after HER debate -- above any of the men in just-completed #GOPdebate says she's on …   
4  RT @DanScavino: #GOPDebate w/ @realDonaldTrump delivered the highest ratings in the history of presidential debates. #Trump2016 http://t.co…   
5       RT @GregAbbott_TX: @TedCruz: "On my first day I will rescind every illegal executive action taken by Barack Obama." #GOPDebate @FoxNews   
6  RT @warriorwoman91: I liked her and was happy when I heard she was going to be the moderator. Not anymore. #GOPDebate @megynkelly  https://…   

  sentiment  
1  Positive  
3  Positive  
4  Positive  
5  Positive  
6  Negative  


Negative    8493
Positive    2236
Name: sentiment, dtype: int64

In [16]:
data.head()

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…,Positive
3,RT @RobGeorge: That Carly Fiorina is trending -- hours after HER debate -- above any of the men in just-completed #GOPdebate says she's on …,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump delivered the highest ratings in the history of presidential debates. #Trump2016 http://t.co…,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day I will rescind every illegal executive action taken by Barack Obama."" #GOPDebate @FoxNews",Positive
6,RT @warriorwoman91: I liked her and was happy when I heard she was going to be the moderator. Not anymore. #GOPDebate @megynkelly https://…,Negative


In [17]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub("[^a-zA-Z0-9\s]", '', x))
data['text']

1               rt scottwalker didnt catch the full gopdebate last night here are some of scotts best lines in 90 seconds walker16 httptcozsff
3           rt robgeorge that carly fiorina is trending  hours after her debate  above any of the men in justcompleted gopdebate says shes on 
4             rt danscavino gopdebate w realdonaldtrump delivered the highest ratings in the history of presidential debates trump2016 httptco
5                rt gregabbotttx tedcruz on my first day i will rescind every illegal executive action taken by barack obama gopdebate foxnews
6           rt warriorwoman91 i liked her and was happy when i heard she was going to be the moderator not anymore gopdebate megynkelly  https
                                                                         ...                                                                  
13866              rt cappyyarbrough love to see men who will never be faced with a pregnancy talk about what i can do with my body  gopdebate

In [18]:
for _,row in data.iterrows():
    text = row[0]
    row[0] = row[0].replace('rt','')
    temp = ""
    for word in row[0].split():
        if not word.startswith('http'):
            temp+= word + ' '
    row[0] = temp

In [19]:
data.head(10)

,text,sentiment
1,scottwalker didnt catch the full gopdebate last night here are some of scotts best lines in 90 seconds walker16,Positive
3,robgeorge that carly fiorina is trending hours after her debate above any of the men in justcompleted gopdebate says shes on,Positive
4,danscavino gopdebate w realdonaldtrump delivered the highest ratings in the history of presidential debates trump2016,Positive
5,gregabbotttx tedcruz on my first day i will rescind every illegal executive action taken by barack obama gopdebate foxnews,Positive
6,warriorwoman91 i liked her and was happy when i heard she was going to be the moderator not anymore gopdebate megynkelly,Negative
8,deer in the headlights lizzwinstead ben carson may be the only brain surgeon who has performed a lobotomy on himself gopdebate,Negative
9,nancyosborne180 last nights debate proved it gopdebate batsask badassteachersa tbats,Negative
10,jgreendc realdonaldtrump in all fairness billclinton owns that phrasegopdebate,Negative
11,waynedupreeshow just woke up to tweet this out gopdebate best line of the night via govmikehuckabee,Positive
12,me reading my familys comments about how great the gopdebate was,Negative


In [33]:
data.to_csv('data/cleaned_text.csv')

In [20]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [40]:
tok_json = tokenizer.to_json()
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tok_json, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 28, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [23]:
batch_size = 27
model.fit(X_train, Y_train, epochs = 9, batch_size=batch_size, verbose = 2)

2022-12-15 13:56:39.777026: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/9
267/267 - 36s - loss: 0.4313 - accuracy: 0.8191
Epoch 2/9
267/267 - 34s - loss: 0.3194 - accuracy: 0.8649
Epoch 3/9
267/267 - 32s - loss: 0.2772 - accuracy: 0.8851
Epoch 4/9
267/267 - 32s - loss: 0.2434 - accuracy: 0.9011
Epoch 5/9
267/267 - 33s - loss: 0.2221 - accuracy: 0.9101
Epoch 6/9
267/267 - 33s - loss: 0.1934 - accuracy: 0.9215
Epoch 7/9
267/267 - 33s - loss: 0.1710 - accuracy: 0.9342
Epoch 8/9
267/267 - 33s - loss: 0.1535 - accuracy: 0.9377
Epoch 9/9
267/267 - 33s - loss: 0.1433 - accuracy: 0.9416


In [24]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

76/76 - 2s - loss: 0.5890 - accuracy: 0.8413
score: 0.59
acc: 0.84


In [25]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(validation_size):
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    clear_output(wait = True)
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 59.8705501618123 %
neg_acc 91.68765743073047 %


In [26]:
model.save("model")

2022-12-15 14:05:45.300662: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model/assets
